# Managing External Resources with LangChain Agents
- **LLMs** are powerful but struggle with **rule-based tasks**, such as:
  - **Mathematical calculations**
  - **Logical reasoning**
  - **Complex decision-making**
- **LangChain Agents** enhance LLMs by interacting with:
  - **Search engines**
  - **Calculators**
  - **APIs**
  - **Databases**
- **Agents** can run code, retrieve information, and **sequence actions dynamically**.

- The following mechanisms use basic chains:
- **LLMChain**: A simple chain that returns responses based on **user queries** and **retrieved context**.
- **RouterChain**: A more complex chain that selects a **prompt template** based on **user input**.
- **LangChain Agents**:
  - Act as **reasoning engines** that **decide** which tools to use.
  - Execute **actions**, such as **retrieving data from APIs** or **performing calculations**.

## **LangChain Agent Features**
1. **Provides tools/toolkits** (sets of functions for different actions).
2. Uses the **ReAct (Reasoning + Acting) framework** to select the appropriate tool.
3. Repeats:
   - **Decides an action** based on the query.
   - **Runs the action**.
   - **Observes the result**.
   - **Continues until the stopping condition is met**.

### **Built-in Tools**
- **LLMMathChain**: Used for mathematical computations.
- **Custom Tools**: Create **API calls** or **database lookups** for specific tasks.

This example demonstrates how an agent:
- Uses the **ReAct framework**.
- Calls the **LLMMathChain** tool for **math calculations**.
- Passes the computed result to the **LLM** for final processing.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain
from langchain_aws import ChatBedrock
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

# Initialize Chat Model
chat = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": 0.1})

# Define ReAct Prompt Template
prompt_template = """Answer the following questions as best you can.
You have access to the following tools:\n\n{tools}\n\n
Use the following format:\n\nQuestion: {input}\n
Thought: {agent_scratchpad}
"""

# Initialize LLM for agent and math chain
react_agent_llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")
math_chain_llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

# Load Tools
tools = load_tools([], llm=react_agent_llm)

# Initialize LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

# Define Math Prompt
llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
{{single line mathematical expression that solves the problem}}
```

Assistant:
Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}
Assistant:"""

tools.append(
    Tool.from_function(
         func=llm_math_chain.run,
         name="Calculator",
         description="Useful for when you need to answer questions about math.",
    )
)

react_agent = create_react_agent(react_agent_llm,
    tools,
    PromptTemplate.from_template(prompt_template)
         # max_iteration=2,
         # return_intermediate_steps=True,
         # handle_parsing_errors=True,
    )

agent_executor = AgentExecutor(
agent=react_agent,
tools=tools,
verbose=True,
handle_parsing_errors=True,
max_iterations = 10 # useful when agent is stuck in a loop
)

agent_executor.invoke({"input": "What is the distance between San Francisco and Los Angeles? If I travel from San Francisco to Los Angeles with the speed of 40MPH how long will it take to reach?"})

## **Key Takeaways**
- **LLMs alone cannot handle** **mathematics, logic, and API calls** effectively.
- **LangChain Agents** enhance LLMs by integrating **external tools and APIs**.
- **ReAct (Reasoning + Acting) framework** helps agents choose **relevant tools** dynamically.
- **LLMMathChain** enables **accurate mathematical computations**.
- **LangGraph** is the **next-generation runtime** for LangChain agents.